In [9]:
from ib_async import *
import requests, pytz
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import pandas_market_calendars as mcal
from zoneinfo import ZoneInfo
import yfinance as yf
from data_and_research import ac

In [10]:
def get_last_full_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = ZoneInfo('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now()
    
    # Ensure current_datetime is timezone-aware
    if current_datetime.tzinfo is None:
        current_datetime = current_datetime.replace(tzinfo=ZoneInfo('UTC'))
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for the current day and the previous day
    schedule = nyse.schedule(start_date=nyse_time.date() - timedelta(days=1), end_date=nyse_time.date())
    
    if not schedule.empty:
        last_close = schedule.iloc[-1]['market_close'].astimezone(nyse_tz)
        
        # If current time is after the last close, that day is the last full trading day
        if nyse_time >= last_close:
            return last_close.date()
        else:
            # Otherwise, we need to find the previous trading day
            previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
            return previous_trading_days[-1].date()
    else:
        # If there's no schedule for today and yesterday (weekend or holiday), 
        # find the last trading day
        previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
        return previous_trading_days[-1].date()

In [13]:
get_last_full_trading_day()

datetime.date(2024, 9, 30)

In [11]:
def get_current_or_next_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = ZoneInfo('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now()
    
    # Ensure current_datetime is timezone-aware
    if current_datetime.tzinfo is None:
        current_datetime = current_datetime.replace(tzinfo=ZoneInfo('UTC'))
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for today and the next few days
    schedule = nyse.schedule(start_date=nyse_time.date(), end_date=nyse_time.date() + timedelta(days=10))
    
    if not schedule.empty:
        market_open = schedule.iloc[0]['market_open'].astimezone(nyse_tz)
        market_close = schedule.iloc[0]['market_close'].astimezone(nyse_tz)
        
        # If the market is currently open, return today as the current trading day
        if market_open <= nyse_time <= market_close:
            return nyse_time.date()
        else:
            # Otherwise, find the next trading day
            next_trading_days = nyse.valid_days(start_date=nyse_time.date() + timedelta(days=1), end_date=nyse_time.date() + timedelta(days=10))
            return next_trading_days[0].date()
    else:
        # If no schedule for today (market holiday), find the next trading day
        next_trading_days = nyse.valid_days(start_date=nyse_time.date(), end_date=nyse_time.date() + timedelta(days=10))
        return next_trading_days[0].date()

In [14]:
get_current_or_next_trading_day()

datetime.date(2024, 10, 1)

In [15]:
def get_earnings():
    # Get the last full trading day and the current or next trading day
    last_trading_day = get_last_full_trading_day()
    next_trading_day = get_current_or_next_trading_day()

    # Define the SQL query with the specific dates and conditions
    query = f"""
    WITH LatestEarnings AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY act_symbol ORDER BY `date` DESC) AS rn
        FROM `earnings_calendar`
        WHERE `when` IS NOT NULL
    )
    SELECT *
    FROM LatestEarnings
    WHERE rn = 1
    AND (
        (`date` = '{last_trading_day}' AND `when` = 'After market close') OR
        (`date` = '{next_trading_day}' AND `when` = 'Before market open')
    )
    ORDER BY `act_symbol` ASC;
    """
    
    # URL encode the query
    encoded_query = requests.utils.quote(query)
    
    # Set the DoltHub API endpoint and parameters
    endpoint = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/earnings/master?q={encoded_query}"
    
    # Make the API request
    response = requests.get(endpoint)
    response.raise_for_status()  # Check for errors

    # Check the content of the response
    data = response.json()

    # Handle the response and convert to a pandas DataFrame if successful
    if data.get('query_execution_status') == 'Success':
        df = pd.DataFrame(data['rows'])
        df = df.rename(columns={'act_symbol': 'symbol'})
        df = df[['symbol', 'date', 'when']]
        return df
    else:
        print(f"Query Error: {data.get('query_execution_message')}")

In [16]:
get_earnings()

,symbol,date,when
0,AYI,2024-10-01,Before market open
1,MKC,2024-10-01,Before market open
2,MKC.V,2024-10-01,Before market open
3,PAYX,2024-10-01,Before market open
4,POCI,2024-09-30,After market close
5,SUUN,2024-09-30,After market close
6,TRAK,2024-09-30,After market close
7,UNFI,2024-10-01,Before market open
8,VRAR,2024-09-30,After market close


In [17]:
def get_vol_data(symbols: list[str] = None, curated = True, include_yf = True):

    # Build the query to get the max date from the volatility_history table
    query_max_date = """
    SELECT MAX(`date`) AS max_date
    FROM `volatility_history`
    """
    encoded_query_max_date = requests.utils.quote(query_max_date)
    endpoint_max_date = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_max_date}"
    response_max_date = requests.get(endpoint_max_date) # Make the request to DoltHub for the max date
    response_max_date.raise_for_status()  # Check for errors

    # Convert the response to a pandas DataFrame
    data_max_date = response_max_date.json()

    # Extract the maximum date from the response
    if data_max_date.get('query_execution_status') == 'Success' and data_max_date.get('rows'):
        max_date = data_max_date['rows'][0]['max_date']
        print("Looking up vol data for:", max_date)
    else:
        print(f"Query Error: {data_max_date.get('query_execution_message')}")
        max_date = None

    if not max_date:
        return None
    
    if not symbols:
        earnings_table = get_earnings()
        symbols = earnings_table['symbol'].tolist()

    # Build the query to get the vol data for the max date
    symbols_str = ', '.join(f"'{symbol}'" for symbol in symbols)
    
    query_volatility = f"""
    SELECT *
    FROM `volatility_history`
    WHERE `date` = '{max_date}'
    AND `act_symbol` IN ({symbols_str})
    ORDER BY `act_symbol` ASC;
    """
    encoded_query_volatility = requests.utils.quote(query_volatility)
    endpoint_volatility = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_volatility}"
    response_volatility = requests.get(endpoint_volatility) # Make the request to DoltHub for volatility history
    response_volatility.raise_for_status()  # Check for errors

    # Convert the response to a pandas DataFrame
    data_volatility = response_volatility.json()

    # Check and convert the response to a pandas DataFrame
    if data_volatility.get('query_execution_status') == 'RowLimit':
        if data_volatility.get('rows'):
            df_volatility = pd.DataFrame(data_volatility['rows'])
        else:
            print("No data found for the maximum date.")
    else:
        df_volatility = pd.DataFrame(data_volatility['rows'])
    
    if not curated:
        return df_volatility
    
    df_vol = df_volatility[['act_symbol', 'date', 'hv_current', 'iv_current']].astype({'hv_current': 'float','iv_current': 'float'})
    df_vol['vol_premium'] = df_vol['iv_current']/df_vol['hv_current']

    if include_yf:
        end_date = datetime.now().date()
        start_date = end_date - timedelta(days=365*2)
        yf_data = yf.download(df_vol['act_symbol'].tolist(), start=start_date, end=end_date)
        
        # Calculate daily returns
        daily_returns = yf_data['Close'].pct_change()
        
        # Calculate 30-day rolling volatility
        rolling_volatility = daily_returns.rolling(window=30).std() * np.sqrt(252)
        
        # Get the most recent volatility for each symbol
        latest_volatility = rolling_volatility.iloc[-1]
        
        # Merge the calculated volatility and close price with df_vol
        df_vol = df_vol.merge(
            pd.DataFrame({
                'calculated_volatility': latest_volatility,
                'close': yf_data['Close'].iloc[-1]
            }),
            left_on='act_symbol',
            right_index=True
        )
        # Calculate vol_premium using the calculated volatility
        df_vol['vol_premium'] = df_vol['iv_current'] / df_vol['calculated_volatility']

    return df_vol.sort_values(by='vol_premium', ascending=False)



In [25]:
df = get_vol_data()

Looking up vol data for: 2024-09-27


[*********************100%***********************]  4 of 4 completed


In [26]:
df

,act_symbol,date,hv_current,iv_current,vol_premium,calculated_volatility,close
1,MKC,2024-09-27,0.1156,0.2667,2.302754,0.115818,83.239998
2,PAYX,2024-09-27,0.1403,0.3107,2.209338,0.140630,132.990005
0,AYI,2024-09-27,0.2300,0.4497,1.954993,0.230026,270.000000
3,UNFI,2024-09-27,0.4331,0.7526,1.725950,0.436050,16.730000


In [22]:
def connect_to_IB(port=7497, clientid=0, symbol=None):
    util.startLoop()  # Needed in script mode
    ib = IB()
    try:
        ib.connect('127.0.0.1', port, clientId=clientid)
    except ConnectionError:
        ib = None  # Reset ib on failure
    return ib

In [23]:
ib = connect_to_IB()

In [43]:
ib.reqMarketDataType(4)


In [44]:
symbols =df.act_symbol.tolist()
symbol = symbols[0]
symbol

'MKC'

In [45]:
stk = Stock(symbol, exchange='NYSE', currency='USD')

In [46]:
ib.qualifyContracts(stk)

[Stock(conId=271556, symbol='MKC', exchange='NYSE', primaryExchange='NYSE', currency='USD', localSymbol='MKC', tradingClass='MKC')]

In [47]:
[ticker] = ib.reqTickers(stk)

In [48]:
ticker

Ticker(contract=Stock(conId=271556, symbol='MKC', exchange='NYSE', primaryExchange='NYSE', currency='USD', localSymbol='MKC', tradingClass='MKC'), time=datetime.datetime(2024, 9, 30, 21, 29, 25, 347205, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=82.3, bidSize=1.0, ask=82.5, askSize=3.0, last=82.3, lastSize=22.0, volume=32141.0, open=83.15, high=83.22, low=81.92, close=83.24, bboExchange='a60001', snapshotPermissions=4)

In [52]:
stk_price =ticker.marketPrice()

In [50]:
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,BOX,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
1,CBOE2,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
2,MIAX,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
3,CBOE,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
4,MERCURY,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
5,EDGX,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
6,PEARL,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
7,GEMINI,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
8,ISE,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."
9,NASDAQBX,271556,MKC,100,"[20241018, 20241115, 20241220, 20250117, 20250...","[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65...."


In [51]:
chain = next(c for c in chains if c.tradingClass == stk.symbol and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='271556', tradingClass='MKC', multiplier='100', expirations=['20241018', '20241115', '20241220', '20250117', '20250321', '20250620', '20250919', '20260116', '20270115'], strikes=[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125.0])

In [61]:
strikes = [strike for strike in chain.strikes
        if stk_price - 20 < strike < stk_price + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P']

contracts = [Option(stk.symbol, expiration, strike, right, 'SMART', tradingClass=stk.symbol)
        for right in rights
        for expiration in expirations
        for strike in strikes]

In [62]:
contracts = ib.qualifyContracts(*contracts)


In [63]:
contracts

[Option(conId=723116428, symbol='MKC', lastTradeDateOrContractMonth='20241018', strike=65.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='MKC   241018P00065000', tradingClass='MKC'),
 Option(conId=723116505, symbol='MKC', lastTradeDateOrContractMonth='20241018', strike=70.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='MKC   241018P00070000', tradingClass='MKC'),
 Option(conId=723116591, symbol='MKC', lastTradeDateOrContractMonth='20241018', strike=75.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='MKC   241018P00075000', tradingClass='MKC'),
 Option(conId=723116687, symbol='MKC', lastTradeDateOrContractMonth='20241018', strike=80.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='MKC   241018P00080000', tradingClass='MKC'),
 Option(conId=723116774, symbol='MKC', lastTradeDateOrContractMonth='20241018', strike=85.0, right='P', multiplier='100', exchange='SMART', curr